http://berkeleyearth.org/data/
pip install xarray[complete]

In [1]:
import xarray as xr
import pandas as pd
from tqdm import tqdm
from functools import reduce


/home/nlztrk/anaconda3/envs/wildfire/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
LAT_RANGE = (36, 42)
LON_RANGE = (26, 45)

In [3]:
data_files_min = [
                '../data/raw_data/Complete_TMIN_Daily_LatLong1_2010.nc',
                '../data/raw_data/Complete_TMIN_Daily_LatLong1_2020.nc',
             ]

data_files_max = [
                '../data/raw_data/Complete_TMAX_Daily_LatLong1_2010.nc',
                '../data/raw_data/Complete_TMAX_Daily_LatLong1_2020.nc',
             ]

data_files_avg = [
                '../data/raw_data/Complete_TAVG_Daily_LatLong1_2010.nc',
                '../data/raw_data/Complete_TAVG_Daily_LatLong1_2020.nc',
             ]

suffixes = ["_min", "_max", "_avg"]

In [5]:
ds = xr.open_dataset(data_files_min[0])
ds

<xarray.Dataset>
Dimensions:      (longitude: 360, latitude: 180, time: 3652, day_number: 365)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Dimensions without coordinates: time, day_number
Data variables:
    date_number  (time) float64 ...
    year         (time) float64 ...
    month        (time) float64 ...
    day          (time) float64 ...
    day_of_year  (time) float64 ...
    land_mask    (latitude, longitude) float64 ...
    temperature  (time, latitude, longitude) float32 ...
    climatology  (day_number, latitude, longitude) float32 ...
Attributes:
    Conventions:     Berkeley Earth Internal Convention (based on CF-1.5)
    title:           Gridded Berkeley Earth Surface Temperature Anomaly Field
    history:         06-Dec-2021 01:12:04
    institution:     Berkeley Earth Surface Temperature Project
    source_history:  05-Dec-2021 20:15:45
    comment:         This file contains surface temperature anomaly field gri...

In [4]:
temperature_dfs = []


for data_i, data_files in enumerate(tqdm([data_files_min, data_files_max, data_files_avg])):
    datasets = []

    for data_file in data_files:
        ds = xr.open_dataset(data_file)

        climatology_df = ds[["climatology"]]
        climatology_df = climatology_df.where((climatology_df.latitude >= LAT_RANGE[0]) &\
                          (climatology_df.latitude <= LAT_RANGE[1]) &\
                          (climatology_df.longitude >= LON_RANGE[0]) &\
                          (climatology_df.longitude <= LON_RANGE[1]),
                      drop=True)\
                      .to_dataframe().reset_index().dropna().rename(columns={"day_number":"day_of_year"})   
        climatology_df = climatology_df.groupby(["climatology", "longitude", "latitude"]).mean().reset_index()

        ds = ds[["temperature", "month", "year", "day_of_year"]]
        ds = ds.where((ds.latitude >= LAT_RANGE[0]) &\
                      (ds.latitude <= LAT_RANGE[1]) &\
                      (ds.longitude >= LON_RANGE[0]) &\
                      (ds.longitude <= LON_RANGE[1]),
                      drop=True)\
                      .to_dataframe().reset_index().dropna().drop(labels=["time", ], axis=1)
        ds = ds.groupby(["latitude", "longitude", "month", "year"]).mean().reset_index()

        climatology_df = climatology_df.merge(ds, how="left", on=["day_of_year", "latitude", "longitude"])\
                .drop(labels=["day_of_year"], axis=1)

        climatology_df["temperature"] += climatology_df["climatology"] 
        climatology_df.drop(labels=["climatology"], axis=1, inplace=True)
        climatology_df.rename(columns={"temperature":"temperature"+suffixes[data_i]}, inplace=True)  
        
        datasets.append(climatology_df.dropna())

    datasets = pd.concat(datasets, ignore_index=True)    
    temperature_dfs.append(datasets)

  0%|                                                     | 0/3 [00:00<?, ?it/s]/home/nlztrk/anaconda3/envs/wildfire/lib/python3.8/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.20s/it]


In [5]:
full_temp_df = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              on=["latitude", "longitude", "month", "year"], 
                                              how='left'), 
                  temperature_dfs)

In [6]:
full_temp_df.to_csv("../data/processed_data/temperatures.csv", index=False)